In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten, add, Dropout
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
import os


Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
%cd '/content/gdrive/My Drive/DLCB'
!ls

/content/gdrive/My Drive/DLCB
mnist_autoencoder.h5  NguyenVanPhat.h5


In [0]:
batch_size = 128
epochs = 200
data_augmentation = True
num_classes = 10
num_filters = 16
kernel_size = 3
n = 5

In [7]:
(X_train, Y_train), (x_test, y_test) = cifar10.load_data()
 
x_train, y_train = X_train[:40000, :], Y_train[:40000]
x_val, y_val = X_train[40000: 50000, :], Y_train[40000: 50000,]
x_train, x_val, x_test = x_train.astype('float32')/255, x_val.astype('float32')/255, x_test.astype('float32')/255

y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

input_shape = x_train.shape[1:]

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
print(y_train[1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [0]:
def resnet_block(x, num_filter=num_filters, increase=False):
    stride = 1

    if increase:
        stride = 2

    y1 = Activation('relu')(BatchNormalization()(x))
    conv_1 = Conv2D(filters=num_filter, kernel_size=3, strides=stride, padding='same', kernel_initializer="he_normal")(y1)
    y2 = Activation('relu')(BatchNormalization()(conv_1))
    conv_2 = Conv2D(filters=num_filter, kernel_size=3, strides=1, padding='same', kernel_initializer="he_normal")(y2)

    if increase:
        y = Conv2D(filters=num_filter, kernel_size=1, strides=stride, padding='same', kernel_initializer="he_normal")(y1)
        block = add([conv_2, y])
    else:
        block = add([conv_2, x])
    return block

In [0]:
inputs = Input(shape=input_shape)
x = Conv2D(filters=num_filters, kernel_size=3, strides=1, padding='same', kernel_initializer="he_normal")(inputs)

for _ in range(n):
    x = resnet_block(x, 16, False)


x = resnet_block(x, 32, True)
for _ in range(1, n):
    x = resnet_block(x, 32, False)


x = resnet_block(x, 64, True)
for _ in range(1, n):
    x = resnet_block(x, 64, False)
    
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = AveragePooling2D(pool_size=8)(x)
x = Flatten()(x)

outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)


In [0]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [0]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [0]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
datagen.fit(x_train)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

callbacks = [lr_reducer, lr_scheduler]

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), validation_data=(x_val, y_val), epochs=epochs, verbose=True,
                  workers=4, shuffle=True, callbacks=callbacks)

Using real-time data augmentation.
Epoch 1/200
313/313 [==============================] - 63s 201ms/step - loss: 1.5686 - acc: 0.4188 - val_loss: 1.6242 - val_acc: 0.4291
Epoch 2/200
313/313 [==============================] - 48s 154ms/step - loss: 1.2204 - acc: 0.5618 - val_loss: 1.9158 - val_acc: 0.4385
Epoch 3/200
313/313 [==============================] - 49s 155ms/step - loss: 1.0632 - acc: 0.6224 - val_loss: 1.2914 - val_acc: 0.5732
Epoch 4/200
313/313 [==============================] - 48s 155ms/step - loss: 0.9378 - acc: 0.6705 - val_loss: 1.2224 - val_acc: 0.5725
Epoch 5/200
313/313 [==============================] - 48s 154ms/step - loss: 0.8502 - acc: 0.6999 - val_loss: 1.0769 - val_acc: 0.6317
Epoch 6/200
313/313 [==============================] - 48s 154ms/step - loss: 0.7875 - acc: 0.7231 - val_loss: 0.9854 - val_acc: 0.6744
Epoch 7/200
313/313 [==============================] - 48s 154ms/step - loss: 0.7251 - acc: 0.7479 - val_loss: 0.9266 - val_acc: 0.6952
Epoch 8/200
3

In [0]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


In [0]:
model.save('NguyenVanPhat.h5')

In [8]:
# Code load model to test
from keras.models import load_model
model = load_model('NguyenVanPhat.h5')
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


10000/10000 [==============================] - 8s 838us/step
Test loss: 0.5453910734206439
Test accuracy: 0.8906
